In [1]:
import socket

После того как мы проимпортировали с вами модуль `socket`, мы можем создать новый сокет.

- `socket.socket(family=AF_INET, type=SOCK_STREAM)`
    Функция получает 2 аргумента:
    - `family` - семейство адресов
    - `type` - тип сокета
- `socket.AF_INET` - ipv4 адрес
- `socket.SOCK_STREAM` - `TCP` сокет

- `sock` - объект сокета

In [2]:
# sjck
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

После того как мы создали сокет(`sock`), нам следует его настроить. По умолчанию, даже если, сокет был закрыт верно_(о чем мы поговорим позже)_, мы не сможем сразу же открыть новый сокет на том же порту, от лица другого процесса.

Это значит, что открыть новый сокет в том же процессе я смогу, но как только я перезапущу свой скрипт_(создам новый процесс)_, зарезервировать ранее используемый порт будет невозможно некоторое время.

Для избежания подобного поведения, мы делаем следующую операцию:

In [3]:
sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

А теперь давайте, привяжем сокет к адресу, для этого у сокета надо вызвать метод `bind` с одним аргументом(адресом, который мы планируем слушать)

In [4]:
sock.bind(('0.0.0.0', 9999))

Осталась буквально одна операция - мы должны начать получать соединения, для этого вызовем метод `listen` у сокета.

`socket.listen` получает один аргумет - это кол-во соединений в очереди на обработку.

In [5]:
sock.listen(5)

### Получение данных из сокета

Теперь, после того как сокет настроен мы можем принимать соединения от клиентов. Кода в следующей ячейке будет побольше, и его следует объяснить.

- В первую очередь мы стартуем бесконечный цикл
- Внутри цикла мы будем получать соединения используя метод `sock.accept()`
  
  `sock.accept()` остановит работу интерпретатора до тех пор, пока соединение не будет получено, как только мы получим соединения из метода вернется кортеж с 2 элементами:
  - `conn` - сокет для общения с клиентом
  - `addr` - адрес клиента `(ip, port)`
- После получения соединения(`conn`) мы должны начать получение данных из этого сокета, для этого вызовем метод `conn.recv`

  `conn.recv()` получает сколько байт мы хотим прочитать, а точнее какой объем памяти под эти данные мы выделяем, естественно, что прочитав `1024` байтика мы можем и не получить весь объем информации которую отправляет нам клиент.
  
  Для того чтобы получить всю информацию, мы обернем операцию получения данных в цикл и как только все данные из сокета будут получены, мы выйдем из цикла с помощью инструкции `break`

In [ ]:
while True:
    conn, addr = sock.accept()
    print(f"New connection from client: {addr}")
    data = bytearray()
    while True:
        received_data = conn.recv(1024)
        data += received_data
        # следующая строчка может показаться вам непонятной... оооокей.. как бы
        # выходим из цикла если получили ноль байт, но что значит проверка длины получнных данных?
        # в действительности все просто, если полученных данных меньше чем размер буфера(1024) - это значит
        # что все данные от клиента мы уже получили и следующего чанка(куска данных) ожидать не стоит
        if not received_data or len(received_data) < 1024:
            break
    print(data)
    conn.close()

Отлично! мне уже нравится что мы написали, мы можем получать данные от клиента, но давайте сделаем следующее:

- Мы будем получать от клиента картинки и сохранять их на диск
  
  Для отправки изображения вы можете воспользоваться следующей командой:
  ```bash
  cat <someimage.jpeg> | nc localhost 9999
  ```
- Есть некоторая проблемка - мы не можем определить разширение файла полученного через команду выше, поэтому мы доверимся клиенту и будем считать, что он оправляет нам изображения в формате `JPEG`.
- Имя для файла с картинкой мы будем получать используя модуль `uuid`.

In [ ]:
import os
import uuid

def get_random_filename():
    """Функция вернет случайный путь до файла с картинкой в временной папке"""
    return os.path.join(os.getenv("TMPDIR"), f"{uuid.uuid4().hex}.jpeg")

while True:
    conn, addr = sock.accept()
    file_path = get_random_filename()
    print(f"File will be saved as: {file_path}")
    with open(file_path, "wb") as f:
        while True:
            chunk = conn.recv(1024)
            f.write(chunk)
            if not chunk or len(chunk) < 1024:
                break
    conn.close()

File will saved as: /var/folders/n4/6psz406s401cd4w7j0sly1gh0000gn/T/8a0f759ebb32410a8d3eb99c14a1183f.jpeg
File will saved as: /var/folders/n4/6psz406s401cd4w7j0sly1gh0000gn/T/1cef6e90b347480d8b9cfe72e8cb89b9.jpeg
